In [ ]:
import os
import yaml
import time
import random
from pathlib import Path

import pandas as pd
import numpy as np
import torch
import scanpy as sc

import STAGATE_pyG

In [ ]:
adata1 = sc.read_h5ad('../../data/xenium/breast/data1_split.h5ad')
adata2 = sc.read_h5ad('../../data/visium/breast/data3_split.h5ad')
adata1.obs['dataset'] = 'dataset1'
adata2.obs['dataset'] = 'dataset2'

In [ ]:
STAGATE_pyG.Cal_Spatial_Net(adata1, k_cutoff=20, model='KNN')
STAGATE_pyG.Cal_Spatial_Net(adata2, k_cutoff=20, model='KNN')
adata = sc.concat([adata1, adata2])
adata.uns['Spatial_Net'] = pd.concat([adata1.uns['Spatial_Net'], adata2.uns['Spatial_Net']])
# sc.pp.highly_variable_genes(adata, flavor="seurat_v3", n_top_genes=3000)
# sc.pp.normalize_total(adata, target_sum=1e4)
# sc.pp.log1p(adata)

sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata)
adata = STAGATE_pyG.train_STAGATE(adata)

In [ ]:
embd0 = adata[adata.obs['dataset']=='dataset1'].obsm['STAGATE']
embd1 = adata[adata.obs['dataset']=='dataset2'].obsm['STAGATE']

In [ ]:
np.savetxt('./stagate_emb0.csv', embd0, delimiter=',')
np.savetxt('./stagate_emb1.csv', embd1, delimiter=',')